In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

In [2]:
from datetime import datetime
from datetime import timedelta

In [3]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

In [4]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
from geopy.distance import geodesic as GD

In [5]:
coord = pd.read_excel('data/terminal_data_hackathon v4.xlsx', sheet_name='TIDS', index_col=0)
incomes = pd.read_excel('data/terminal_data_hackathon v4.xlsx', sheet_name='Incomes', index_col='TID')
times = pd.read_csv('data/times v4.csv')

In [6]:
coord.shape

(1630, 2)

In [9]:
#максимально допустимая сумма денег в терминале 
max_cash = 1000000
# % на остаток в терминале
p_per_day = 0.02/365

# издержки на обслуживание одного терминала
def service_cost(cash, p_service_cost=0.0001):
    return max(p_service_cost*cash, 100)

#максимально допустимое время, в течение которого терминал можно не обслуживать 
days_limit = 14
#стоимость одного броневика на день 
car_cost = 20000
#рабочий день в минутах
work_hours = 12*60
#среднее время простоя/обслуживания
service_time = 10
#затраты на броневик за минуту
car_cost_per_minute = car_cost/work_hours

# Анализ входных данных

In [78]:
(incomes['остаток на 31.08.2022 (входящий)'] > max_cash).sum()

11

In [79]:
incomes[incomes['остаток на 31.08.2022 (входящий)'] > max_cash]

,остаток на 31.08.2022 (входящий),2022-09-01 00:00:00,2022-09-02 00:00:00,2022-09-03 00:00:00,2022-09-04 00:00:00,2022-09-05 00:00:00,2022-09-06 00:00:00,2022-09-07 00:00:00,2022-09-08 00:00:00,2022-09-09 00:00:00,2022-09-10 00:00:00,2022-09-11 00:00:00,2022-09-12 00:00:00,2022-09-13 00:00:00,2022-09-14 00:00:00,2022-09-15 00:00:00,2022-09-16 00:00:00,2022-09-17 00:00:00,2022-09-18 00:00:00,2022-09-19 00:00:00,2022-09-20 00:00:00,2022-09-21 00:00:00,2022-09-22 00:00:00,2022-09-23 00:00:00,2022-09-24 00:00:00,2022-09-25 00:00:00,2022-09-26 00:00:00,2022-09-27 00:00:00,2022-09-28 00:00:00,2022-09-29 00:00:00,2022-09-30 00:00:00,2022-10-01 00:00:00,2022-10-02 00:00:00,2022-10-03 00:00:00,2022-10-04 00:00:00,2022-10-05 00:00:00,2022-10-06 00:00:00,2022-10-07 00:00:00,2022-10-08 00:00:00,2022-10-09 00:00:00,2022-10-10 00:00:00,2022-10-11 00:00:00,2022-10-12 00:00:00,2022-10-13 00:00:00,2022-10-14 00:00:00,2022-10-15 00:00:00,2022-10-16 00:00:00,2022-10-17 00:00:00,2022-10-18 00:00:00,2022-10-19 00:00:00,2022-10-20 00:00:00,2022-10-21 00:00:00,2022-10-22 00:00:00,2022-10-23 00:00:00,2022-10-24 00:00:00,2022-10-25 00:00:00,2022-10-26 00:00:00,2022-10-27 00:00:00,2022-10-28 00:00:00,2022-10-29 00:00:00,2022-10-30 00:00:00,2022-10-31 00:00:00,2022-11-01 00:00:00,2022-11-02 00:00:00,2022-11-03 00:00:00,2022-11-04 00:00:00,2022-11-05 00:00:00,2022-11-06 00:00:00,2022-11-07 00:00:00,2022-11-08 00:00:00,2022-11-09 00:00:00,2022-11-10 00:00:00,2022-11-11 00:00:00,2022-11-12 00:00:00,2022-11-13 00:00:00,2022-11-14 00:00:00,2022-11-15 00:00:00,2022-11-16 00:00:00,2022-11-17 00:00:00,2022-11-18 00:00:00,2022-11-19 00:00:00,2022-11-20 00:00:00,2022-11-21 00:00:00,2022-11-22 00:00:00,2022-11-23 00:00:00,2022-11-24 00:00:00,2022-11-25 00:00:00,2022-11-26 00:00:00,2022-11-27 00:00:00,2022-11-28 00:00:00,2022-11-29 00:00:00,2022-11-30 00:00:00
TID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
608039,1132000,156000,103000,120000,105000,130000,121000,105000,140000,146000,112000,115000,125000,133000,124000,130000,123000,126000,100000,108000,115000,125000,125000,145000,100000,113000,133000,105000,106000,123000,131000,121000,100000,90000,123000,112000,137000,126000,108000,95000,117000,100000,120000,115000,103000,115000,95000,105000,95000,80000,115000,100000,100000,116000,102000,110000,100000,126000,115000,113000,103000,135000,108000,105000,134000,121000,115000,125000,115000,112000,131000,79000,137000,125000,113000,100000,120000,125000,118000,97000,105000,104000,113000,100000,120000,115000,120000,125000,102000,120000,116000,105000
630465,1053000,6000,2000,4000,15000,18000,11000,9000,0,10000,3000,15000,5000,2000,22000,15000,10000,28000,13000,5000,2000,1000,1000,10000,12000,28000,5000,19000,3000,0,24000,0,0,6000,13000,8000,6000,2000,7000,3000,7000,8000,1000,0,5000,13000,5000,1000,0,0,31000,6000,13000,21000,6000,0,10000,0,8000,0,0,12000,1000,3000,7000,0,5000,1000,14000,7000,4000,0,0,6000,3000,0,52000,0,0,23000,3000,2000,9000,0,15000,11000,13000,8000,0,0,0,0
634558,1517000,510000,589000,518000,522000,512000,450000,456000,495000,481000,495000,461000,491000,431000,504000,468000,459000,448000,482000,466000,446000,466000,532000,414000,501000,476000,484000,484000,505000,460000,500000,514000,546000,448000,395000,533000,547000,506000,436000,445000,378000,513000,457000,440000,449000,512000,469000,414000,406000,380000,503000,422000,484000,521000,485000,420000,243000,561000,433000,600000,455000,520000,413000,503000,489000,585000,519000,497000,439000,414000,429000,433000,472000,499000,248000,748000,384000,396000,458000,424000,505000,574000,506000,364000,373000,440000,461000,495000,441000,424000,399000,381000
635763,1010000,85000,57000,102000,95000,68000,69000,75000,68000,73000,75000,80000,70000,45000,50000,46000,87000,120000,75000,70000,45000,30000,50000,80000,68000,85000,45000,35000,47000,57000,43000,100000,75000,33000,63000,60000,42000,48000,60000,80000,55000,28000,52000,45000,63000,117000,40000,25000,55000,50000,52000,62

In [87]:
test = (incomes > max_cash).sum()
test[test > 0]

остаток на 31.08.2022 (входящий)    11
2022-09-03 00:00:00                  1
2022-09-08 00:00:00                  1
2022-09-14 00:00:00                  1
2022-09-20 00:00:00                  1
2022-09-21 00:00:00                  1
2022-09-26 00:00:00                  1
2022-09-27 00:00:00                  1
2022-10-01 00:00:00                  1
2022-10-14 00:00:00                  1
2022-10-20 00:00:00                  1
2022-10-25 00:00:00                  1
2022-11-25 00:00:00                  1
dtype: int64

__Есть терминалы с остатком более 1 млн. В приростах также есть такие случаи. Уточнить.__

In [88]:
incomes[(incomes.sum(axis=1)/max_cash /(91/days_limit)) < 1]

,остаток на 31.08.2022 (входящий),2022-09-01 00:00:00,2022-09-02 00:00:00,2022-09-03 00:00:00,2022-09-04 00:00:00,2022-09-05 00:00:00,2022-09-06 00:00:00,2022-09-07 00:00:00,2022-09-08 00:00:00,2022-09-09 00:00:00,2022-09-10 00:00:00,2022-09-11 00:00:00,2022-09-12 00:00:00,2022-09-13 00:00:00,2022-09-14 00:00:00,2022-09-15 00:00:00,2022-09-16 00:00:00,2022-09-17 00:00:00,2022-09-18 00:00:00,2022-09-19 00:00:00,2022-09-20 00:00:00,2022-09-21 00:00:00,2022-09-22 00:00:00,2022-09-23 00:00:00,2022-09-24 00:00:00,2022-09-25 00:00:00,2022-09-26 00:00:00,2022-09-27 00:00:00,2022-09-28 00:00:00,2022-09-29 00:00:00,2022-09-30 00:00:00,2022-10-01 00:00:00,2022-10-02 00:00:00,2022-10-03 00:00:00,2022-10-04 00:00:00,2022-10-05 00:00:00,2022-10-06 00:00:00,2022-10-07 00:00:00,2022-10-08 00:00:00,2022-10-09 00:00:00,2022-10-10 00:00:00,2022-10-11 00:00:00,2022-10-12 00:00:00,2022-10-13 00:00:00,2022-10-14 00:00:00,2022-10-15 00:00:00,2022-10-16 00:00:00,2022-10-17 00:00:00,2022-10-18 00:00:00,2022-10-19 00:00:00,2022-10-20 00:00:00,2022-10-21 00:00:00,2022-10-22 00:00:00,2022-10-23 00:00:00,2022-10-24 00:00:00,2022-10-25 00:00:00,2022-10-26 00:00:00,2022-10-27 00:00:00,2022-10-28 00:00:00,2022-10-29 00:00:00,2022-10-30 00:00:00,2022-10-31 00:00:00,2022-11-01 00:00:00,2022-11-02 00:00:00,2022-11-03 00:00:00,2022-11-04 00:00:00,2022-11-05 00:00:00,2022-11-06 00:00:00,2022-11-07 00:00:00,2022-11-08 00:00:00,2022-11-09 00:00:00,2022-11-10 00:00:00,2022-11-11 00:00:00,2022-11-12 00:00:00,2022-11-13 00:00:00,2022-11-14 00:00:00,2022-11-15 00:00:00,2022-11-16 00:00:00,2022-11-17 00:00:00,2022-11-18 00:00:00,2022-11-19 00:00:00,2022-11-20 00:00:00,2022-11-21 00:00:00,2022-11-22 00:00:00,2022-11-23 00:00:00,2022-11-24 00:00:00,2022-11-25 00:00:00,2022-11-26 00:00:00,2022-11-27 00:00:00,2022-11-28 00:00:00,2022-11-29 00:00:00,2022-11-30 00:00:00
TID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
406148,355000,50000,73000,53000,65000,75000,100000,53000,52000,82000,63000,50000,55000,85000,83000,50000,55000,85000,57000,70000,90000,80000,85000,75000,55000,67000,75000,75000,65000,62000,76000,70000,68000,75000,75000,65000,60000,90000,75000,0,0,108000,51000,68000,0,143000,60000,60000,56000,82000,70000,69000,85000,70000,61000,60000,65000,62000,75000,56000,85000,55000,75000,50000,53000,75000,62000,50000,70000,80000,95000,50000,50000,125000,65000,50000,63000,58000,65000,71000,87000,67000,48000,55000,65000,85000,95000,68000,62000,0,118000,70000
406536,651000,63000,65000,0,0,201000,80000,133000,68000,60000,86000,74000,58000,67000,70000,71000,140000,74000,66000,69000,67000,81000,71000,124000,61000,57000,0,132000,67000,73000,112000,72000,67000,83000,74000,112000,65000,61000,67000,62000,0,126000,70000,67000,139000,61000,45000,65000,66000,61000,60000,114000,57000,0,99000,63000,44000,48000,87000,58000,39000,0,57000,58000,50000,0,93000,0,79000,120000,46000,49000,47000,40000,0,71000,50000,100000,55000,37000,48000,30000,40000,34000,79000,50000,43000,36000,34000,31000,32000,39000
605355,247000,46000,58000,40000,32000,44000,0,40000,36000,45000,48000,46000,35000,46000,54000,47000,39000,25000,0,94000,35000,48000,48000,47000,35000,37000,53000,46000,34000,45000,64000,46000,46000,0,90000,44000,47000,56000,46000,49000,42000,44000,48000,53000,44000,31000,56000,52000,27000,56000,49000,50000,43000,37000,46000,42000,43000,53000,53000,50000,41000,44000,40000,40000,65000,36000,43000,41000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
605360,191000,64000,51000,55000,49000,60000,55000,63000,39000,60000,59000,58000,69000,31000,61000,56000,64000,45000,65000,0,0,0,0,0,0,75000,55000,63000,78000,64000,45000,38000,67000,56000,55000,75000,52000,58000,63000,57000,68000,70000,62000,65000,78000,37000,60000,44000,65000,55000,76000,56000,66000,50000,61000,65000,67000,62000,63000,64000,53000,67000,55000,60000,64000,79000,54000,48000,69000,47000,58000,75000,65000,77000,52000,62000,58000,65000,52000,62000,60000,66000,65000,58000,44000,76000,49000,60000,5

In [95]:
1214/1630

0.7447852760736197

__1214 терминалов из 1630 (почти 75%) "переполняются" в среднем реже, чем раз в 14 дней. 
Т.е. мы должны их обсуживаьть исключительно из условий ТЗ (максимально допустимое время, в течение которого терминал можно не обслуживать).__

In [74]:
times

,Origin_tid,Destination_tid,Total_Time,Total_time_plus_service
0,636538,683103,15.32,25.32
1,636538,634763,16.20,26.20
2,636538,683128,16.27,26.27
3,636538,683789,16.77,26.77
4,636538,634709,17.67,27.67
...,...,...,...,...
2655265,619736,646840,50.69,60.69
2655266,688241,648534,43.02,53.02
2655267,646076,637536,35.62,45.62
2655268,648791,608006,126.87,136.87


In [107]:
times['Total_time_plus_service'] = times.Total_Time+service_time

In [126]:
def approx_time_cost(x, top=days_limit*2): #days_limit*2 тк считаем от терминала i до терминала X и от X до терминала j 
    return (x.sort_values()[:top]).mean()

In [127]:
#(В ИТОГОМ РЕШЕНИИ НЕ ИСПОЛЬЗУЕМ)

# посчитаем приблизительное время от+до терминала (как усредненное время до N ближайших точек)
#+время на обслуживание для дальнейших попыток ранжирований
time_cost = times.groupby('Origin_tid').agg({'Total_Time':approx_time_cost}).rename(
    columns={'Total_Time':'approx_time_cost'})/2 + times.groupby('Destination_tid').agg({'Total_Time':approx_time_cost}).rename(
    columns={'Total_Time':'approx_time_cost'})/2 + service_time
time_cost['approx_time_cost_rub'] = time_cost.approx_time_cost*car_cost_per_minute

In [128]:
time_cost.approx_time_cost_rub.describe()

count    1630.000000
mean      618.921785
std       247.374155
min       383.630952
25%       452.938988
50%       523.881448
75%       700.798611
max      2305.496032
Name: approx_time_cost_rub, dtype: float64

__С учетом того, что у нас нет данных о выездах за 2 недели ДО 01.09 (начинаем смотреть выполнение 2 условия с 14.09 и до 30.11)__

In [129]:
# сколько в среднем потребуется объезжать точек(терминалов) за 14 дней 
#с учетом ограничений ТЗ - 1)переполняемость и 2) мин раз в 14 дней
(incomes.sum(axis=1)/max_cash /(91/days_limit)).apply(np.ceil).sum()

2120.0

In [130]:
# сколько потребуется объезжать точек(терминалов) за первые 14 дней 
#с учетом ограничений ТЗ - 1)переполняемость и 2) мин раз в 14 дней
(incomes.iloc[:,:15].sum(axis=1)/max_cash /(14/days_limit)).apply(np.ceil).sum()

2621.0

In [131]:
time_cost.approx_time_cost.describe()

count    1630.000000
mean       22.281184
std         8.905470
min        13.810714
25%        16.305804
50%        18.859732
75%        25.228750
max        82.997857
Name: approx_time_cost, dtype: float64

In [135]:
time = 22.28

In [136]:
# по первым 14 дням
2621/days_limit*time/720 # приблизительная оценка - минимальное кол-во машин

5.793242063492064

In [137]:
# после старта работы алгоритмов и наличии данных о предыдущих выездах за последние 14 дней
2120/days_limit*time/720 # приблизительная оценка - минимальное кол-во машин

4.685873015873016

In [138]:
# для 4 машин
4*720/(2621/days_limit) - service_time

5.383441434566958

In [139]:
# для 4 машин
4*720/(2120/days_limit) - service_time

9.018867924528305

In [140]:
# для 5 машин
5*720/(2621/days_limit) - service_time

9.229301793208698

In [141]:
# для 3 машин - среднее время в дороге 1.5 мин это невозможно=)
3*720/(2621/days_limit) - service_time

1.5375810759252193

__Удастся отработать на 4 машинах ЕСЛИ среднее время между точками будет менее 5,4 мин на всей выборке и 9 мин впоследствие.__
__Будем сначала работать с всей выборкой, т.к. нам важно показать, что выбранного числа броневиков хватит на любых днях из предоставленной выборки__ 

__Предварительный вывод - 4-5 машин это минимум!__
__4 - если темпы прироста наличности будут "удачно" синхронизированы с расстояниями между точками__

In [35]:
max_cash*0.0001 # стоимость обслуживания исходя из условий всегда 100р, т.к. при текущих лимитах 100р это максимум. 

100.0

In [36]:
max_cash*2/100/365 # максимальный % на остаток

54.794520547945204

__Обслуживая терминал мы всегда в минусе. Это больше необходимость__

# Идея 1.0 Равномерно распределить точки для обсуживания

In [113]:
add_points=23 #подбираемый параметр
balance=pd.DataFrame(data=0, index=incomes.index, columns = incomes.columns[1:])
balance.insert(loc=0, column='остаток на 31.08.2022 (входящий)', value=incomes['остаток на 31.08.2022 (входящий)'])
predict_cash_trust = 0.9
start_num_points_base = int(np.ceil(incomes.shape[0]/days_limit))
start_num_points = start_num_points_base+add_points
all_terms_set = set(balance.index)
print(f'Всего точек {start_num_points}')

d=1
koef = 1
extra_points={}
extra_points_count=0
terms_dict_0 ={0:{}}

day_df_start = pd.DataFrame(time_cost.approx_time_cost_rub*(-1))
day_df_start.index.name ='TID'

while d < balance.shape[1]:
    day_predict = incomes.iloc[:,d]
    day_df = day_df_start.copy()
    day_df['pred_balance'] = balance.iloc[:,d-1] + day_predict*(2-predict_cash_trust)
    day_df['priority'] = (day_df.pred_balance >= max_cash).apply(lambda x: 0 if x==True else -1)
        
    day_df['cost_per_service'] = - day_df.pred_balance.apply(service_cost)
    day_df['cash_rate_per_day'] = day_df.pred_balance*p_per_day
    day_df['all_costs'] = day_df.approx_time_cost_rub + day_df.cost_per_service + day_df.cash_rate_per_day
    day_df = day_df.dropna()
    
    x=days_limit-1
    while x > 0:
        day_df.loc[list(set(day_df.index)&set(terms_dict_0[max(d-x,0)])), 'nes_degree'] = x
        x-=1
    day_df['nes_degree'] = day_df['nes_degree'].fillna(days_limit)
    
    day_df['all_costs_cor'] = day_df['all_costs']*(koef**day_df['nes_degree'])
    #day_df['all_costs_cor'] = day_df['all_costs']+(koef*day_df['nes_degree'])
        
    if d >=days_limit:
        #not_served = list(balance.index[(balance.iloc[:,max(1,d-days_limit+1):d] == 0).sum(axis=1) == 0])
        #print(not_served_yet)
        not_served = list(day_df[(day_df['nes_degree'] == days_limit)].index)
        day_df.loc[not_served, 'priority'] = 0
    
    day_df = day_df.sort_values(by=['priority', 'all_costs_cor'], ascending=False)
    
    balance.iloc[:,d] = day_predict # ???
    points = day_df.priority.value_counts()[0]
    
    if points <= start_num_points:
        terms = list(day_df.index[start_num_points:])
    else:
        extra_points[d] = points
        extra_points_count+=1
        print('День '+str(balance.columns[d]))
        print(f'Экстремально много точек для обслуживания - {points} (более {start_num_points})')
        print('______________________________________________________________')
        terms = list(day_df.index[points:])
    
    balance.loc[terms, balance.columns[d]] = day_df.loc[terms, 'pred_balance']        
    terms_dict_0[d] = all_terms_set - set(terms)
    d+=1


Всего точек 140
День 2022-09-14 00:00:00
Экстремально много точек для обслуживания - 729 (более 140)
______________________________________________________________
День 2022-09-28 00:00:00
Экстремально много точек для обслуживания - 717 (более 140)
______________________________________________________________
День 2022-10-12 00:00:00
Экстремально много точек для обслуживания - 715 (более 140)
______________________________________________________________
День 2022-10-26 00:00:00
Экстремально много точек для обслуживания - 709 (более 140)
______________________________________________________________
День 2022-11-09 00:00:00
Экстремально много точек для обслуживания - 715 (более 140)
______________________________________________________________
День 2022-11-23 00:00:00
Экстремально много точек для обслуживания - 704 (более 140)
______________________________________________________________


__Вывод: слишком много невыгодных точек откладывается на 14/28/тд дни. Нужно их "размазать" по всему сроку. Для этого приоритет выезда должен зависеть от параметра вроде "кол-во дней, которые терминал не обсуживался" - nes_degree__

In [117]:
add_points=23 # можно тюнить число доп точек
balance=pd.DataFrame(data=0, index=incomes.index, columns = incomes.columns[1:])
balance.insert(loc=0, column='остаток на 31.08.2022 (входящий)', value=incomes['остаток на 31.08.2022 (входящий)'])
predict_cash_trust = 0.9
car_cost_per_minute = car_cost/work_hours
start_num_points_base = int(np.ceil(incomes.shape[0]/days_limit))
start_num_points = start_num_points_base+add_points
all_terms_set = set(balance.index)
print(f'Всего точек {start_num_points}')

d=1
koef = 0.6
extra_points={}
extra_points_count=0
terms_dict_0 ={0:{}}

day_df_start = pd.DataFrame(time_cost.approx_time_cost_rub*(-1))
day_df_start.index.name ='TID'

while d < balance.shape[1]:
    day_predict = incomes.iloc[:,d]
    day_df = day_df_start.copy()
    day_df['pred_balance'] = balance.iloc[:,d-1] + day_predict*(2-predict_cash_trust)
    day_df['priority'] = (day_df.pred_balance >= max_cash).apply(lambda x: 0 if x==True else -1)
        
    day_df['cost_per_service'] = - day_df.pred_balance.apply(service_cost)
    day_df['cash_rate_per_day'] = day_df.pred_balance*p_per_day
    day_df['all_costs'] = day_df.approx_time_cost_rub + day_df.cost_per_service + day_df.cash_rate_per_day
    day_df = day_df.dropna()
    
    x=days_limit-1
    while x > 0:
        day_df.loc[list(set(day_df.index)&set(terms_dict_0[max(d-x,0)])), 'nes_degree'] = x
        x-=1
    day_df['nes_degree'] = day_df['nes_degree'].fillna(days_limit) # кол-во дней, которые терминал не обсуживался
    
    day_df['all_costs_cor'] = day_df['all_costs']*(koef**day_df['nes_degree'])
    #day_df['all_costs_cor'] = day_df['all_costs']+(koef*day_df['nes_degree'])
        
    if d >=days_limit:
        #not_served = list(balance.index[(balance.iloc[:,max(1,d-days_limit+1):d] == 0).sum(axis=1) == 0])
        #print(not_served_yet)
        not_served = list(day_df[(day_df['nes_degree'] == days_limit)].index)
        day_df.loc[not_served, 'priority'] = 0
    
    day_df = day_df.sort_values(by=['priority', 'all_costs_cor'], ascending=False)
    
    balance.iloc[:,d] = day_predict # ???
    points = day_df.priority.value_counts()[0]
    
    if points <= start_num_points:
        terms = list(day_df.index[start_num_points:])
    else:
        extra_points[d] = points
        extra_points_count+=1
        print('День '+str(balance.columns[d]))
        print(f'Экстремально много точек для обслуживания - {points} (более {start_num_points})')
        print('______________________________________________________________')
        terms = list(day_df.index[points:])
    
    balance.loc[terms, balance.columns[d]] = day_df.loc[terms, 'pred_balance']        
    terms_dict_0[d] = all_terms_set - set(terms)
    d+=1


Всего точек 140
День 2022-10-12 00:00:00
Экстремально много точек для обслуживания - 146 (более 140)
______________________________________________________________


# Идея 1.1. Временные затраты как стоимость добавления точек к уже построенному плану по обязательным точкам объезда

__Попробуем также посчитать временные затраты на заезд не как усредненное по N ближайшим точкам, а как стоимость добавления этих точек к уже построенному графу из терминалов с Приоритетом 0 (обязательно заехать)__

In [118]:
%%time
add_points=26 # можно тюнить число доп точек
balance=pd.DataFrame(data=0, index=incomes.index, columns = incomes.columns[1:])
balance.insert(loc=0, column='остаток на 31.08.2022 (входящий)', value=incomes['остаток на 31.08.2022 (входящий)'])
day_df_start = pd.DataFrame()
predict_cash_trust = 0.9
car_cost_per_minute = car_cost/work_hours
start_num_points_base = int(np.ceil(incomes.shape[0]/days_limit))
start_num_points = start_num_points_base+add_points
print(f'Всего точек {start_num_points}')
all_terms_set = set(balance.index)

d=1
koef = 0.55
extra_points={}
extra_points_count=0
terms_dict ={0:{}}

while d < balance.shape[1]:
    
    day_predict = incomes.iloc[:,d]
    day_df = day_df_start.copy()
    day_df['pred_balance'] = balance.iloc[:,d-1] + day_predict*(2-predict_cash_trust)
    day_df['priority'] = (day_df.pred_balance >= max_cash).apply(lambda x: 0 if x==True else -1)
        
    day_df['cost_per_service'] = - day_df.pred_balance.apply(service_cost)
    day_df['cash_rate_per_day'] = day_df.pred_balance*p_per_day
    
    time_cost_new = times[times.Destination_tid.isin(list(day_df[day_df['priority'] == 0].index))].groupby(
    'Origin_tid').agg({'Total_Time':approx_time_cost}).rename(
    columns={'Total_Time':'approx_time_cost'}
    ) + times[times.Origin_tid.isin(list(day_df[day_df['priority'] == 0].index))].groupby('Destination_tid').agg({'Total_Time':approx_time_cost}).rename(
    columns={'Total_Time':'approx_time_cost'}) + service_time
    time_cost_new['approx_time_cost_rub'] = -time_cost_new.approx_time_cost*car_cost_per_minute
    
    day_df= day_df.merge(time_cost_new, how='left', right_index=True, left_index=True)
    
    day_df['all_costs'] = day_df.approx_time_cost_rub + day_df.cost_per_service + day_df.cash_rate_per_day
    #day_df = day_df.dropna()
    
    x=days_limit-1
    while x > 0:
        day_df.loc[list(set(day_df.index)&set(terms_dict[max(d-x,0)])), 'nes_degree'] = x
        x-=1
    day_df['nes_degree'] = day_df['nes_degree'].fillna(days_limit)
    
    day_df['all_costs_cor'] = day_df['all_costs']*(koef**day_df['nes_degree'])
    #day_df['all_costs_cor'] = day_df['all_costs']+(koef*day_df['nes_degree'])
        
    if d >=days_limit:
        not_served = list(day_df[(day_df['nes_degree'] == days_limit)].index)
        day_df.loc[not_served, 'priority'] = 0
    
    day_df = day_df.sort_values(by=['priority', 'all_costs_cor'], ascending=False)
    
    balance.iloc[:,d] = day_predict # ???
    points = day_df.priority.value_counts()[0]
    
    if points <= start_num_points:
        terms = list(day_df.index[start_num_points:])
    else:
        extra_points[d] = points
        extra_points_count+=1
        print('День '+str(balance.columns[d]))
        print(f'Экстремально много точек для обслуживания - {points} (более {start_num_points})')
        print('______________________________________________________________')
        terms = list(day_df.index[points:])
    
    balance.loc[terms, balance.columns[d]] = day_df.loc[terms, 'pred_balance']        
    terms_dict[d] = all_terms_set - set(terms)
    d+=1


Всего точек 143
День 2022-10-12 00:00:00
Экстремально много точек для обслуживания - 158 (более 143)
______________________________________________________________
День 2022-10-26 00:00:00
Экстремально много точек для обслуживания - 145 (более 143)
______________________________________________________________
Wall time: 1min 6s


In [119]:
extra_points_count

2

Теперь посмотрим сколько броневиков нам хватает. Цель - уменьшить их до 4/5.

In [124]:
def create_data_model(start_route_point_pos, num_veh=5):
    data = {}
    data['distance_matrix'] = distance_matrix
    data['num_vehicles'] = num_veh
    data['depot'] = start_route_point_pos
    return data

def distance_callback(from_index, to_index):
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data['distance_matrix'][from_node][to_node]

In [125]:
#coord_sample = coord.loc[list(terms_dict[l])]
#center_lon = coord_sample.longitude.mean()
#center_lat = coord_sample.latitude.mean()
#
#coord_sample['dist_to_center_point'] = coord_sample.apply(
#    lambda row: GD((row['longitude'], row['latitude']), (center_lon,center_lat)).km, axis=1)
#start_route_point = coord_sample.sort_values(by='dist_to_center_point').index[0]
#start_route_point_pos = list(distance_matrix_df.index).index(start_route_point)

In [176]:
#imput_matrix = [list(map(lambda x: int(np.ceil(x)), row)) for row in distance_matrix]

In [184]:
def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f'Objective: {solution.ObjectiveValue()}')
    max_route_distance = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        while not routing.IsEnd(index):
            plan_output += ' {} -> '.format(manager.IndexToNode(index))
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += '{}\n'.format(manager.IndexToNode(index))
        plan_output += 'Distance of the route: {}min\n'.format(route_distance)
        print(plan_output)
        max_route_distance = max(route_distance, max_route_distance)
    print('Maximum of the route distances: {}min'.format(max_route_distance))
    return max_route_distance

__Проверим, удасться ли уложиться в отведенное время на 6 машинах. Начальная точка - искусственная с близостью 0__

In [623]:
route_times={}
for i in range(1,30):
    road = terms_dict[i]
    sample = times[(times.Origin_tid.isin(road))&(times.Destination_tid.isin(road))]
    sample.loc[-1]=[0,0,0,0]
    distance_matrix_df = pd.pivot_table(sample, values='Total_time_plus_service', index=['Origin_tid'],
                       columns=['Destination_tid'], aggfunc=np.sum).fillna(0).apply(np.round).applymap(int)
    distance_matrix = np.array(distance_matrix_df)
    
    #coord_sample = coord.loc[list(terms_dict[i])]
    #center_lon = coord_sample.longitude.mean()
    #center_lat = coord_sample.latitude.mean()
    #
    #coord_sample['dist_to_center_point'] = coord_sample.apply(
    #    lambda row: GD((row['longitude'], row['latitude']), (center_lon,center_lat)).km, axis=1)
    #start_route_point = coord_sample.sort_values(by='dist_to_center_point').index[0]
    #start_route_point_pos = list(distance_matrix_df.index).index(start_route_point)
    
    data = create_data_model(start_route_point_pos=0, num_veh=6)
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                       data['num_vehicles'],
                                       data['depot']
                                      )
    routing = pywrapcp.RoutingModel(manager)
    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    
    dimension_name = 'Time'
    routing.AddDimension(
    transit_callback_index,
    0,  # no slack
    work_hours-service_time,  # vehicle maximum travel distance
    True,  # start cumul to zero
    dimension_name)
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    distance_dimension.SetGlobalSpanCostCoefficient(100)
    
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    search_parameters.time_limit.seconds = 60
    
    solution = routing.SolveWithParameters(search_parameters)
    # Print solution on console.
    if solution:
        route_times[i]=print_solution(data, manager, routing, solution)
        print(f'{i}day - Done')
    else:
        print('No solution found !')
        route_times[i] = np.nan

D:\Programs\Anaconda\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


Objective: 37087
Route for vehicle 0:
 0 ->  93 ->  66 ->  106 ->  122 ->  137 ->  21 ->  142 ->  65 ->  60 ->  20 ->  114 ->  64 ->  9 ->  25 ->  71 ->  99 ->  136 ->  140 ->  123 ->  12 ->  13 ->  11 ->  139 ->  100 ->  130 ->  67 ->  3 -> 0
Distance of the route: 350min

Route for vehicle 1:
 0 ->  35 ->  18 ->  116 ->  6 ->  47 ->  138 ->  135 ->  118 ->  52 ->  31 ->  84 ->  134 ->  121 ->  132 ->  128 ->  16 ->  98 ->  143 ->  94 ->  26 -> 0
Distance of the route: 347min

Route for vehicle 2:
 0 ->  54 ->  126 ->  120 ->  91 ->  34 ->  53 ->  111 ->  30 ->  92 ->  15 ->  46 ->  10 ->  124 ->  59 ->  39 ->  44 ->  22 ->  41 ->  45 ->  2 ->  33 -> 0
Distance of the route: 349min

Route for vehicle 3:
 0 ->  129 ->  72 ->  79 ->  38 ->  96 ->  50 ->  68 ->  28 ->  57 ->  95 ->  58 ->  115 ->  17 ->  113 ->  29 ->  7 ->  19 ->  1 ->  105 ->  110 ->  109 -> 0
Distance of the route: 346min

Route for vehicle 4:
 0 ->  131 ->  63 ->  27 ->  102 ->  61 ->  24 ->  48 ->  88 ->  80 ->  103

D:\Programs\Anaconda\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


Objective: 42682
Route for vehicle 0:
 0 ->  57 ->  16 ->  19 ->  66 ->  94 ->  100 ->  140 ->  52 ->  18 ->  13 ->  118 ->  84 ->  11 ->  102 ->  32 ->  58 ->  136 ->  89 ->  83 ->  42 ->  69 ->  36 ->  71 ->  24 ->  73 ->  43 ->  25 ->  111 ->  55 -> 0
Distance of the route: 403min

Route for vehicle 1:
 0 ->  76 ->  127 ->  142 ->  105 ->  139 ->  122 ->  33 ->  61 ->  59 ->  93 ->  110 ->  30 ->  129 ->  15 ->  114 ->  53 ->  138 ->  108 ->  14 ->  1 ->  126 ->  124 ->  123 ->  115 ->  29 -> 0
Distance of the route: 395min

Route for vehicle 2:
 0 ->  46 ->  92 ->  34 ->  95 ->  77 ->  54 ->  7 ->  10 ->  65 ->  17 ->  106 ->  86 ->  41 ->  67 ->  112 ->  23 -> 0
Distance of the route: 401min

Route for vehicle 3:
 0 ->  9 ->  6 ->  113 ->  131 ->  44 ->  8 ->  130 ->  26 ->  128 ->  82 ->  133 ->  50 ->  27 ->  28 ->  134 ->  12 ->  3 ->  141 ->  90 ->  107 ->  87 ->  20 ->  5 ->  116 -> 0
Distance of the route: 403min

Route for vehicle 4:
 0 ->  135 ->  103 ->  101 ->  88 ->  11

D:\Programs\Anaconda\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


...

Objective: 54778
Route for vehicle 0:
 0 ->  28 ->  57 ->  109 ->  54 ->  41 ->  73 ->  122 ->  77 ->  71 ->  6 ->  94 ->  8 ->  9 ->  72 ->  32 ->  61 ->  24 ->  79 ->  31 ->  137 -> 0
Distance of the route: 513min

Route for vehicle 1:
 0 ->  112 ->  141 ->  82 ->  81 ->  118 ->  18 ->  34 ->  102 ->  125 ->  43 ->  22 ->  29 ->  49 ->  123 ->  91 ->  7 ->  95 ->  67 ->  48 ->  139 ->  53 ->  74 ->  20 ->  58 -> 0
Distance of the route: 509min

Route for vehicle 2:
 0 ->  14 ->  135 ->  26 ->  126 ->  87 ->  86 ->  33 ->  80 ->  103 ->  111 ->  4 ->  59 ->  105 ->  143 ->  119 ->  133 ->  120 ->  110 ->  100 ->  63 ->  93 -> 0
Distance of the route: 512min

Route for vehicle 3:
 0 ->  25 ->  37 ->  90 ->  121 ->  69 ->  96 ->  132 ->  129 ->  12 ->  2 ->  23 ->  101 ->  15 ->  5 ->  134 ->  106 ->  117 ->  13 ->  40 ->  113 ->  114 -> 0
Distance of the route: 510min

Route for vehicle 4:
 0 ->  52 ->  44 ->  84 ->  88 ->  83 ->  46 ->  85 ->  89 ->  50 ->  66 ->  78 ->  1 ->  131 -> 

D:\Programs\Anaconda\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


Objective: 67356
Route for vehicle 0:
 0 ->  127 ->  40 ->  1 ->  65 ->  60 ->  105 ->  119 ->  92 ->  39 ->  61 ->  8 ->  7 ->  126 ->  122 ->  121 ->  81 ->  71 ->  68 ->  120 ->  91 ->  12 ->  41 ->  143 -> 0
Distance of the route: 636min

Route for vehicle 1:
 0 ->  130 ->  135 ->  63 ->  79 ->  73 ->  87 ->  128 ->  137 ->  2 ->  85 ->  82 ->  6 ->  5 ->  64 ->  100 ->  66 ->  102 ->  94 ->  3 ->  132 ->  123 ->  53 -> 0
Distance of the route: 629min

Route for vehicle 2:
 0 ->  33 ->  32 ->  116 ->  30 ->  59 ->  138 ->  31 ->  51 ->  77 ->  113 ->  114 ->  109 ->  88 ->  72 ->  4 ->  83 ->  9 ->  10 ->  133 ->  62 ->  101 -> 0
Distance of the route: 628min

Route for vehicle 3:
 0 ->  136 ->  107 ->  99 ->  57 ->  103 ->  34 ->  95 ->  117 ->  111 ->  11 ->  26 ->  25 ->  104 ->  89 ->  46 ->  44 ->  106 ->  142 ->  50 ->  18 ->  20 ->  54 ->  19 ->  16 ->  69 ->  13 ->  15 ->  52 ->  14 ->  55 ->  112 ->  43 ->  78 ->  22 ->  23 ->  28 -> 0
Distance of the route: 635min

Route 

D:\Programs\Anaconda\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


Objective: 47569
Route for vehicle 0:
 0 ->  141 ->  56 ->  77 ->  54 ->  8 ->  116 ->  5 ->  33 ->  41 ->  103 ->  118 ->  49 ->  42 ->  43 ->  142 ->  110 ->  19 ->  4 ->  20 ->  139 -> 0
Distance of the route: 441min

Route for vehicle 1:
 0 ->  105 ->  104 ->  112 ->  69 ->  31 ->  13 ->  16 ->  14 ->  58 ->  129 ->  62 ->  35 ->  61 ->  15 ->  65 ->  64 ->  98 ->  81 ->  90 ->  66 ->  60 ->  80 ->  59 ->  36 -> 0
Distance of the route: 447min

Route for vehicle 2:
 0 ->  88 ->  140 ->  21 ->  34 ->  17 ->  87 ->  101 ->  143 ->  67 ->  37 ->  130 ->  132 ->  133 ->  38 ->  7 ->  10 ->  1 ->  117 ->  78 ->  99 ->  12 ->  2 ->  128 -> 0
Distance of the route: 440min

Route for vehicle 3:
 0 ->  68 ->  106 ->  11 ->  134 ->  121 ->  138 ->  73 ->  119 ->  107 ->  47 ->  44 ->  3 ->  123 ->  28 ->  46 ->  122 ->  74 ->  30 ->  95 ->  111 ->  22 ->  23 ->  45 -> 0
Distance of the route: 445min

Route for vehicle 4:
 0 ->  115 ->  113 ->  102 ->  136 ->  76 ->  97 ->  127 ->  50 ->  83 

In [624]:
len(terms_dict[1])

143

In [625]:
route_times

{1: 350,
 2: 403,
 3: 452,
 4: 440,
 5: 518,
 6: 492,
 7: 544,
 8: 521,
 9: 602,
 10: 571,
 11: 566,
 12: 573,
 13: 537,
 14: 633,
 15: 453,
 16: 451,
 17: 466,
 18: 504,
 19: 533,
 20: 528,
 21: 554,
 22: 589,
 23: 565,
 24: 588,
 25: 577,
 26: 586,
 27: 517,
 28: 636,
 29: 449}

__Отработало с 6 машинами. Время в дороге с большой дисперсией__

__Сравним вариант Идеи 1 с добавления костов за разъезд после основного графа ( он и затратнее) и предварительной усредненной оценки костов__

In [ ]:
route_times_0={}
for i in range(1,30):
    road = terms_dict_0[i]
    sample = times[(times.Origin_tid.isin(road))&(times.Destination_tid.isin(road))]
    sample.loc[-1]=[0,0,0,0]
    distance_matrix_df = pd.pivot_table(sample, values='Total_time_plus_service', index=['Origin_tid'],
                       columns=['Destination_tid'], aggfunc=np.sum).fillna(0).apply(np.round).applymap(int)
    distance_matrix = np.array(distance_matrix_df)
    
    data = create_data_model(start_route_point_pos=0, num_veh=6)
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                       data['num_vehicles'],
                                       data['depot']
                                      )
    routing = pywrapcp.RoutingModel(manager)
    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    
    dimension_name = 'Time'
    routing.AddDimension(
    transit_callback_index,
    0,  # no slack
    work_hours-service_time,  # vehicle maximum travel distance
    True,  # start cumul to zero
    dimension_name)
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    distance_dimension.SetGlobalSpanCostCoefficient(100)
    
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    search_parameters.time_limit.seconds = 60
    
    solution = routing.SolveWithParameters(search_parameters)
    # Print solution on console.
    if solution:
        route_times_0[i]=print_solution(data, manager, routing, solution)
        print(f'{i}day - Done')
    else:
        print('No solution found !')
        route_times_0[i] = np.nan

In [232]:
lst=[]
for i, j in zip(route_times.values(), route_times_0.values()):
    lst.append(i-j)

In [247]:
np.std(list(route_times_0.values()))

50.062101742891066

In [248]:
np.std(list(route_times.values()))

42.6581689912784

In [252]:
np.mean(list(route_times_0.values()))

345.3103448275862

In [253]:
np.mean(list(route_times.values()))

334.9310344827586

In [249]:
np.median(lst)

-11.0

__В итоге лучше показал себя алгоритм добавления костов за разъезд после основного графа (он и затратнее по времени), нежели алгоритм предварительной усредненной оценки костов по N ближайшим точкам.
Начальная точка - искусственная с близостью 0__

__Попробуем 5 машин__

In [291]:
route_times2={}
for i in range(1,30):
    road = terms_dict[i]
    sample = times[(times.Origin_tid.isin(road))&(times.Destination_tid.isin(road))]
    sample.loc[-1]=[0,0,0,0]
    distance_matrix_df = pd.pivot_table(sample, values='Total_time_plus_service', index=['Origin_tid'],
                       columns=['Destination_tid'], aggfunc=np.sum).fillna(0).apply(np.round).applymap(int)
    distance_matrix = np.array(distance_matrix_df)
    
    data = create_data_model(start_route_point_pos=0, num_veh=5)
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                       data['num_vehicles'],
                                       data['depot']
                                      )
    routing = pywrapcp.RoutingModel(manager)
    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    
    dimension_name = 'Time'
    routing.AddDimension(
    transit_callback_index,
    0,  # no slack
    work_hours-service_time,  # vehicle maximum travel distance
    True,  # start cumul to zero
    dimension_name)
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    distance_dimension.SetGlobalSpanCostCoefficient(150)
    
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    search_parameters.time_limit.seconds = 60
    
    solution = routing.SolveWithParameters(search_parameters)
    # Print solution on console.
    if solution:
        route_times2[i]=print_solution(data, manager, routing, solution)
        print(f'{i}day - Done')
    else:
        print('No solution found !')
        route_times2[i] = np.nan

D:\Programs\Anaconda\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


Objective: 65968
Route for vehicle 0:
 0 ->  94 ->  134 ->  116 ->  27 ->  83 ->  37 ->  112 ->  79 ->  24 ->  58 ->  1 ->  136 ->  82 ->  34 ->  113 ->  29 ->  41 ->  138 ->  137 ->  30 ->  76 ->  70 ->  23 ->  56 ->  12 ->  69 ->  119 ->  36 ->  67 ->  48 ->  114 ->  74 -> 0
Distance of the route: 418min

Route for vehicle 1:
 0 ->  65 ->  66 ->  126 ->  6 ->  106 ->  38 ->  11 ->  75 ->  127 ->  50 ->  17 ->  53 ->  31 ->  99 ->  49 ->  54 ->  132 ->  110 ->  128 ->  55 ->  80 ->  73 ->  47 ->  18 ->  21 ->  125 ->  28 ->  3 -> 0
Distance of the route: 426min

Route for vehicle 2:
 0 ->  107 ->  64 ->  40 ->  22 ->  90 ->  5 ->  10 ->  135 ->  123 ->  111 ->  121 ->  115 ->  8 ->  109 ->  7 ->  25 ->  35 -> 0
Distance of the route: 426min

Route for vehicle 3:
 0 ->  108 ->  15 ->  124 ->  39 ->  105 ->  32 ->  120 ->  117 ->  13 ->  77 ->  61 ->  14 ->  81 ->  122 ->  100 ->  9 ->  131 ->  20 ->  16 ->  19 ->  133 ->  118 ->  93 ->  72 ->  51 ->  60 ->  97 ->  104 -> 0
Distance of 

D:\Programs\Anaconda\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


Objective: 87197
Route for vehicle 0:
 0 ->  83 ->  88 ->  52 ->  47 ->  18 ->  63 ->  29 ->  46 ->  136 ->  90 ->  59 ->  133 ->  44 ->  25 ->  105 ->  80 ->  57 ->  82 ->  125 ->  28 ->  67 -> 0
Distance of the route: 563min

Route for vehicle 1:
 0 ->  3 ->  27 ->  5 ->  8 ->  71 ->  39 ->  12 ->  48 ->  2 ->  79 ->  75 ->  43 ->  26 ->  101 ->  138 ->  94 ->  112 ->  92 ->  93 ->  120 ->  96 ->  53 ->  131 ->  115 ->  49 ->  116 ->  135 -> 0
Distance of the route: 556min

Route for vehicle 2:
 0 ->  124 ->  98 ->  117 ->  99 ->  108 ->  129 ->  51 ->  107 ->  54 ->  56 ->  33 ->  103 ->  58 ->  78 ->  45 ->  102 ->  95 ->  42 ->  19 ->  121 ->  31 ->  70 ->  38 ->  37 ->  66 ->  68 ->  128 ->  20 ->  55 ->  137 ->  110 ->  23 ->  81 ->  34 ->  36 ->  62 ->  65 ->  22 ->  106 ->  41 -> 0
Distance of the route: 563min

Route for vehicle 3:
 0 ->  86 ->  64 ->  60 ->  40 ->  14 ->  1 ->  111 ->  4 ->  132 ->  7 ->  104 ->  15 ->  76 ->  126 ->  72 ->  50 ->  87 ->  84 ->  127 ->  85 -

D:\Programs\Anaconda\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


...

Objective: 91423
Route for vehicle 0:
 0 ->  110 ->  109 ->  119 ->  19 ->  38 ->  73 ->  23 ->  41 ->  48 ->  126 ->  61 ->  71 ->  123 ->  69 ->  108 ->  1 ->  100 ->  20 -> 0
Distance of the route: 590min

Route for vehicle 1:
 0 ->  90 ->  31 ->  81 ->  59 ->  113 ->  16 ->  18 ->  87 ->  117 ->  51 ->  105 ->  52 ->  124 ->  36 ->  102 ->  98 -> 0
Distance of the route: 590min

Route for vehicle 2:
 0 ->  118 ->  57 ->  54 ->  60 ->  89 ->  15 ->  14 ->  45 ->  46 ->  29 ->  86 ->  33 ->  122 ->  65 ->  39 ->  30 ->  55 ->  37 ->  68 ->  58 ->  32 ->  25 ->  82 ->  47 ->  43 ->  27 ->  116 ->  107 ->  35 ->  26 ->  66 ->  129 ->  94 ->  111 ->  93 ->  138 ->  74 ->  114 -> 0
Distance of the route: 584min

Route for vehicle 3:
 0 ->  22 ->  115 ->  101 ->  13 ->  136 ->  12 ->  34 ->  104 ->  42 ->  50 ->  83 ->  21 ->  3 ->  4 ->  80 ->  97 ->  24 ->  88 ->  67 ->  28 ->  121 ->  127 ->  49 ->  128 ->  132 ->  125 ->  56 ->  77 ->  72 ->  53 -> 0
Distance of the route: 583min

Rou

D:\Programs\Anaconda\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


No solution found !


D:\Programs\Anaconda\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


Objective: 84147
Route for vehicle 0:
 0 ->  59 ->  76 ->  51 ->  6 ->  28 ->  78 ->  29 ->  86 ->  43 ->  135 ->  46 ->  74 ->  138 ->  71 ->  40 ->  67 ->  22 ->  9 ->  99 ->  92 ->  117 ->  115 ->  36 ->  57 ->  93 ->  20 ->  54 -> 0
Distance of the route: 532min

Route for vehicle 1:
 0 ->  129 ->  38 ->  58 ->  109 ->  55 ->  134 ->  105 ->  128 ->  12 ->  64 ->  101 ->  127 ->  13 ->  48 ->  122 ->  47 ->  110 ->  133 ->  24 ->  118 ->  91 ->  112 ->  119 ->  114 ->  136 ->  116 ->  73 ->  62 ->  39 -> 0
Distance of the route: 542min

Route for vehicle 2:
 0 ->  37 ->  96 ->  94 ->  16 ->  97 ->  1 ->  30 ->  56 ->  100 ->  79 ->  68 ->  75 ->  45 ->  44 ->  27 ->  17 ->  125 ->  70 ->  65 ->  3 ->  137 -> 0
Distance of the route: 543min

Route for vehicle 3:
 0 ->  25 ->  95 ->  104 ->  77 ->  82 ->  83 ->  81 ->  60 ->  10 ->  26 ->  19 ->  111 ->  132 ->  72 ->  49 ->  5 ->  69 ->  41 ->  14 ->  23 ->  33 ->  80 ->  85 ->  106 ->  87 -> 0
Distance of the route: 541min

Route f

In [292]:
route_times2

{1: 426,
 2: 563,
 3: 504,
 4: 555,
 5: 603,
 6: 590,
 7: nan,
 8: 664,
 9: nan,
 10: 633,
 11: nan,
 12: 631,
 13: 566,
 14: nan,
 15: 492,
 16: 552,
 17: 559,
 18: 578,
 19: 623,
 20: 599,
 21: nan,
 22: nan,
 23: nan,
 24: 621,
 25: 639,
 26: 648,
 27: 590,
 28: nan,
 29: 543}

__Не вышло уложиться во время. Хочется, чтобы время в дороге было более однаковым.__

# Идея 2. Со штрафами

In [158]:
def create_data_model(start_route_point_pos, num_veh=5):
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = distance_matrix
    data['num_vehicles'] = num_veh
    data['depot'] = start_route_point_pos
    return data

def distance_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data['distance_matrix'][from_node][to_node]

In [159]:
balance=pd.DataFrame(data=0, index=incomes.index, columns = incomes.columns[1:])
balance.insert(loc=0, column='остаток на 31.08.2022 (входящий)', value=incomes['остаток на 31.08.2022 (входящий)'])
day_df_start = pd.DataFrame(time_cost.approx_time_cost_rub*(-1))
day_df_start.index.name ='TID'

predict_cash_trust = 0.9
car_cost_per_minute = car_cost/work_hours
#start_num_points_base = int(np.ceil(incomes.shape[0]/days_limit))
#start_num_points = start_num_points_base+add_points
all_terms_set = set(balance.index)
#print(f'Всего точек {start_num_points}')

d=1
terms_dict_2 ={0:all_terms_set}
bags_2 = {0:set()}
day_df_dict_2={}
route_times_2={}
route_vehicle_2={}
time_vehicle_2 = {}

In [160]:
min_penalty = np.ceil(times['Total_time_plus_service'].max())

In [161]:
min_car_cost = np.ceil(times['Total_time_plus_service'].min()*20000/12/60)
max_car_cost = np.ceil(times['Total_time_plus_service'].max()*20000/12/60)

In [162]:
100 # максимальные потери при обслуживании
100 - max_cash*2/100/365 # минимальные

45.205479452054796

In [163]:
min_car_cost

278.0

In [165]:
def print_solution(data, manager, routing, solution):
    print(f'DAY {d}')
    print(f'Objective: {solution.ObjectiveValue()}')
    
    dropped_nodes = 'Пропущенные точки:'
    dropped_nodes_lst=[]
    for node in range(routing.Size()):
        if routing.IsStart(node) or routing.IsEnd(node):
            continue
        if solution.Value(routing.NextVar(node)) == node:
            #dropped_nodes += ' {}'.format(manager.IndexToNode(node))
            dropped_nodes_lst.append(manager.IndexToNode(node)-1)
    
    max_route_distance = 0
    route_vehicle = {}
    time_vehicle = {}
    for vehicle_id in range(data['num_vehicles']):
        route_vehicle[vehicle_id] = []
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        while not routing.IsEnd(index):
            plan_output += ' {} -> '.format(manager.IndexToNode(index))
            route_vehicle[vehicle_id].append(manager.IndexToNode(index)-1)
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            r = routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
            route_distance += r
        plan_output += '{}\n'.format(manager.IndexToNode(index))
        plan_output += 'Время по маршруту: {}min\n'.format(route_distance)
        time_vehicle[vehicle_id] = route_distance + 10
        #print(plan_output)
        max_route_distance = max(route_distance, max_route_distance)
    print('Максимальное время по маргруту: {}min'.format(max_route_distance))
    dropped_num = 1630-len(dropped_nodes_lst)
    print(f'Точек объезда в день - {dropped_num}')
    return max_route_distance, dropped_nodes_lst, route_vehicle, time_vehicle

In [166]:
times_c = times.copy()
times_c.loc[-1]=[0,0,0,0]
distance_matrix_df = pd.pivot_table(times_c, values='Total_time_plus_service', index=['Origin_tid'],
                                    columns=['Destination_tid'], aggfunc=np.sum).fillna(0).apply(np.round).applymap(int)
distance_matrix = np.array(distance_matrix_df)
data = create_data_model(start_route_point_pos=0, num_veh=5)
manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                       data['num_vehicles'],
                                       data['depot']
                                      )

In [167]:
distance_matrix_df

Destination_tid,0,406136,406139,406145,406148,406180,406190,406196,406504,406509,406536,406542,406593,406596,406597,407002,407025,605329,605354,605355,605360,605365,605368,605369,605370,605373,605374,605377,605378,605379,605380,605381,605390,605392,605393,605394,606001,606003,606006,606012,606016,606017,606021,606025,606026,606027,606029,606037,606043,606044,...,698955,698956,698957,698959,698960,698964,698965,698968,698970,698971,698972,698978,698983,698984,698985,698988,698989,698991,698993,698994,698998,699004,699005,699022,699046,699048,699052,699054,699059,699060,699068,699070,699074,699088,699090,699091,699095,699098,699099,699449,699451,699540,699560,699572,699577,699578,699579,699629,699641,699664
Origin_tid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
406136,0,0,37,56,66,52,38,53,60,70,63,43,60,46,65,52,71,69,56,58,59,64,57,47,66,46,54,62,54,56,59,55,59,60,70,49,61,62,64,48,51,56,56,69,62,72,59,50,49,62,...,111,109,120,120,124,93,98,105,149,153,124,75,171,100,103,108,109,144,116,117,81,53,58,82,167,69,126,101,93,90,161,75,71,71,81,43,62,63,35,79,65,53,74,63,58,59,61,77,64,58
406139,0,37,0,41,45,35,33,36,45,53,46,29,40,32,49,32,51,48,41,43,44,47,42,31,49,31,37,45,39,41,45,40,44,46,50,34,41,47,44,33,36,41,39,49,47,52,42,35,35,47,...,96,94,100,132,104,105,78,91,131,138,109,55,151,85,88,110,112,129,128,129,61,38,38,62,149,48,112,86,78,75,146,55,56,56,67,28,42,46,21,59,85,36,53,48,40,42,43,62,47,37
406145,0,58,41,0,48,28,44,30,23,34,36,51,53,34,30,45,45,54,24,16,22,30,22,37,28,35,27,26,21,14,17,18,22,18,50,28,52,20,55,30,27,12,26,48,25,49,26,22,32,25,...,74,72,101,138,110,112,79,68,112,116,95,68,157,62,66,123,125,115,134,135,72,20,44,75,130,61,89,71,56,53,131,68,34,34,44,33,51,45,32,58,93,26,42,26,35,33,34,40,37,42
406148,0,69,49,46,0,40,60,43,47,35,35,54,26,50,41,36,18,24,50,47,40,42,50,49,44,50,40,41,47,46,49,41,49,50,19,52,26,47,45,51,50,46,44,17,40,20,41,47,53,40,...,80,78,68,155,79,128,46,63,99,110,120,46,126,88,92,120,122,140,151,152,48,50,43,53,117,54,84,97,82,78,157,46,39,50,70,47,28,31,45,27,108,40,21,51,37,35,36,53,29,40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699578,0,61,42,32,38,19,48,16,40,25,15,52,36,32,21,35,37,44,29,31,33,20,29,28,24,32,18,20,35,32,28,35,28,30,36,31,34,26,56,34,30,32,23,33,31,30,20,36,37,28,...,78,76,91,143,100,116,69,65,104,112,112,57,147,80,83,120,124,132,139,140,62,32,31,64,122,60,86,88,73,70,148,57,36,44,61,36,33,26,33,48,96,19,35,43,18,0,12,47,19,28
699579,0,61,42,34,38,21,48,16,42,27,16,52,36,32,22,35,38,45,31,32,35,21,30,28,26,32,18,21,36,33,29,36,29,31,37,32,34,28,56,36,31,34,25,33,33,31,22,37,38,30,...,80,78,91,143,100,117,69,67,106,114,113,57,147,81,85,120,126,133,139,140,64,33,31,64,124,61,88,90,75,71,150,57,38,46,63,36,34,26,33,48,96,21,37,44,18,12,0,48,21,27
699629,0,79,63,42,55,47,66,49,34,41,49,72,63,56,37,66,52,61,50,43,34,39,48,57,40,57,47,40,43,42,45,37,48,45,57,49,64,44,76,51,49,41,51,55,34,57,46,43,53,34,...,68,66,108,160,117,133,86,67,115,118,116,83,164,84,88,144,146,136,156,157,88,46,64,91,137,83,88,93,78,74,153,84,29,20,52,54,65,59,53,65,114,48,49,40,60,46,48,0,51,62


In [168]:
min_penalty

338.0

In [169]:
data['num_vehicles']

5

In [54]:
d=1

In [170]:
%%time
while d < balance.shape[1]:
    day_predict = incomes.iloc[:,d]
    day_df = day_df_start.copy()
    day_df['pred_balance'] = balance.iloc[:,d-1] + day_predict*(2-predict_cash_trust)
    day_df['priority'] = (day_df.pred_balance >= max_cash).apply(lambda x: 0 if x==True else -1)
        
    day_df['cost_per_service'] = - day_df.pred_balance.apply(lambda x: min(x,max_cash)).apply(service_cost)
    day_df['cash_rate_per_day'] = day_df.pred_balance.apply(lambda x: min(x,max_cash))*p_per_day
    day_df['all_costs'] = day_df.approx_time_cost_rub + day_df.cost_per_service + day_df.cash_rate_per_day
    day_df['costs_without_vehicle'] = day_df.cost_per_service + day_df.cash_rate_per_day
    
    x=days_limit-1
    while x > 0:
        day_df.loc[list(set(day_df.index)&set(terms_dict_2[max(d-x,0)])), 'nes_degree'] = x
        x-=1
    day_df['nes_degree'] = day_df['nes_degree'].fillna(days_limit)
    
    if d >=days_limit:
        #not_served = list(balance.index[(balance.iloc[:,max(1,d-days_limit+1):d] == 0).sum(axis=1) == 0])
        #print(not_served_yet)
        not_served = list(day_df[(day_df['nes_degree'] == days_limit)].index)
        day_df.loc[not_served, 'priority'] = 0
    
    #day_df['penalty'] = ((day_df.priority + 1)*10000000 - min_penalty*(10000/day_df.all_costs)*day_df.nes_degree**(1.5)).apply(int)
    #day_df['penalty'] = (day_df.priority + 1)*1000000 - min_penalty*(10000/day_df.all_costs)*day_df.nes_degree**(1.5)
    #day_df['penalty'] = ((day_df.priority + 1)*10000000 + min_penalty*day_df.nes_degree**(-100/day_df.costs_without_vehicle+2)).apply(int)
    day_df['penalty'] = ((day_df.priority + 1)*10000000 - (10000/day_df.costs_without_vehicle)*day_df.nes_degree**(3.8)).apply(int)
                         
    balance.iloc[:,d] = day_predict 
    points = day_df.priority.value_counts()[0]
    
    routing = pywrapcp.RoutingModel(manager)
    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    
    dimension_name = 'Time'
    routing.AddDimension(
    transit_callback_index,
    0,  # no slack
    work_hours-service_time,  # vehicle maximum travel distance
    True,  # start cumul to zero
    dimension_name)
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    distance_dimension.SetGlobalSpanCostCoefficient(150)
    
    penalty =  day_df['penalty']
    for node, p in zip(range(1, len(data['distance_matrix'])), penalty):
        routing.AddDisjunction([manager.NodeToIndex(node)], p)
    
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
    #search_parameters.time_limit.FromSeconds(1)
    search_parameters.time_limit.seconds = 60
    
    solution = routing.SolveWithParameters(search_parameters)
    # Решение
    if solution:
        route_times_2[d], dropped_ponts, route_vehicle_2[d], time_vehicle_2[d] =print_solution(data, manager, routing, solution)
        print(f'{d} day - Done')
    else:
        print('No solution found !')
        route_times_2[d] = np.nan

    balance.iloc[dropped_ponts, d] = day_df.iloc[dropped_ponts, 1]
    terms_dict_2[d] = all_terms_set - set(day_df.iloc[dropped_ponts].index)
    bags_2[d] = set(day_df[day_df.priority==0].index) - terms_dict_2[d]
    if len(bags_2[d]) > 0:
        print('NOT GOOD')
        day_df_dict_2[d]=day_df
        break
    day_df_dict_2[d]=day_df
    d+=1


DAY 1
Objective: 284617
Максимальное время по маргруту: 708min
Точек объезда в день - 253
1 day - Done
DAY 2
Objective: 2608583
Максимальное время по маргруту: 710min
Точек объезда в день - 238
2 day - Done
DAY 3
Objective: 291790460
Максимальное время по маргруту: 710min
Точек объезда в день - 276
3 day - Done
NOT GOOD
Wall time: 3min


In [171]:
len(bags_2[3])

28

# Далее работа велась через создание класса и тюнинг на ресурсах СберКлауд с 1) увеличением параметра time_limit.seconds и 2) тестом параметров функции штрафов...